# [The Annotated Transformer](http://nlp.seas.harvard.edu/annotated-transformer)

ここ数年の最先端の自然言語処理(Natural Language Processing, NLP)ではTransformerという技術が広く応用されている．
Transformerは[Attention Is All You Need](https://arxiv.org/abs/1706.03762)で提案された手法である．
[The Annotated Transformer](http://nlp.seas.harvard.edu/annotated-transformer)では，注釈付きでこの論文を解説している．
自分のメモ用に，この内容を要約してここに記しておく．

In [3]:
import os
from os.path import exists
import torch
import torch.nn as nn
from torch.nn.functional import log_softmax, pad
import math
import copy
import time
from torch.optim.lr_scheduler import LambdaLR
import pandas as pd
import altair as alt
from torchtext.data.functional import to_map_style_dataset
from torch.utils.data import DataLoader
from torchtext.vocab import build_vocab_from_iterator
import torchtext.datasets as datasets
import spacy
import GPUtil
import warnings
warnings.filterwarnings("ignore")

from torch.utils.data.distributed import DistributedSampler
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP

# Set to False to skip notebook execution (e.g. for debugging)
RUN_EXAMPLES = True

In [4]:
# Some convenience helper functions used throughout the notebook


def is_interactive_notebook():
    return __name__ == "__main__"


def show_example(fn, args=[]):
    if __name__ == "__main__" and RUN_EXAMPLES:
        return fn(*args)


def execute_example(fn, args=[]):
    if __name__ == "__main__" and RUN_EXAMPLES:
        fn(*args)


class DummyOptimizer(torch.optim.Optimizer):
    def __init__(self):
        self.param_groups = [{"lr": 0}]
        None

    def step(self):
        None

    def zero_grad(self, set_to_none=False):
        None


class DummyScheduler:
    def step(self):
        None

## Background

NLPにおいて生じる逐次計算(sequential computation)の演算回数を減らすことが重要である．
かつて提案されたConvS2SやByteNetでは入力する単語数が増加するにつれて，計算量が大幅に増加するという課題があった．
一方で，Transformerでは，Multi-Head Attentionという手法により，計算量を減らすことができる．

## Part 1: Model Architecture

### Model Architecture

NLPにおける[trunsduction model](https://en.wikipedia.org/wiki/Transduction_(machine_learning))(非ラベル化されたデータを教師無し学習によってラベル付けする手法)では，エンコーダ-デコーダ構造(encoder-decoder model structure)がしばしば採用されている．
この手法では，単語列のようなシンボル列$(x_1 ,...,x_n)$を一旦，連続値(z_1,...,z_n)に変換(エンコード)したのち，最終的に(y_1,...,y_n)を出力(デコード)するというものである．
このエンコーダ-デコーダ構造がトップレベルで見た時の構造になるので，ここから見ていく．

In [6]:
class EncoderDecoder(nn.Module):
    """
    A standard Encoder-Decoder architecture. Base for this and many
    other models.
    """

    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator

    def forward(self, src, tgt, src_mask, tgt_mask):
        "Take in and process masked src and target sequences."
        # 入力xをエンコードしてzを生成 -> デコードして出力yを生成
        return self.decode(self.encode(src, src_mask), src_mask, tgt, tgt_mask)

    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask)

    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

In [7]:
class Generator(nn.Module):
    "Define standard linear + softmax generation step."

    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        return log_softmax(self.proj(x), dim=-1)